In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kmeans1/test.csv


In [55]:
df = pd.read_csv('/kaggle/input/kmeans1/test.csv')
df = df[['Article','Summary']]
df

,Article,Summary
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi..."
4,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N..."
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar..."
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...


In [8]:
import nltk
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from scipy.spatial import distance
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

In [9]:
text = df['Article'][0]
text

'Fear shakes Mexico border city after violence leaves 18 deadFear has invaded the Mexican border city of Reynosa after gunmen in vehicles killed 14 people, including taxi drivers, workers and a nursing student, and security forces responded with operations that left four suspects dead.While this city across the border from McAllen, Texas is used to cartel violence as a key trafficking point, the 14 victims in Saturday’s attacks appeared to be what Tamaulipas Gov. Francisco García Cabeza de Vaca called “innocent citizens” rather than members of one gang killed by a rival.Local businessman Misael Chavarria Garza said many businesses closed early Saturday after the attacks and people were very scared as helicopters flew overhead. On Sunday, he said, “the people were quiet as if nothing had happened, but with a feeling of anger because now crime has happened to innocent people.”“It’s not fair,” said taxi driver Rene Guevara, adding that among the dead were two of his fellow taxi drivers wh

In [10]:
sentence = sent_tokenize(text)

In [11]:
corpus = []
for i in range(len(sentence)):
    sen = re.sub('[^a-zA-Z]', " ", sentence[i])  
    sen = sen.lower()                            
    sen=sen.split()                         
    sen = ' '.join([i for i in sen if i not in stopwords.words('english')])   
    corpus.append(sen)

In [12]:
corpus

['fear shakes mexico border city violence leaves deadfear invaded mexican border city reynosa gunmen vehicles killed people including taxi drivers workers nursing student security forces responded operations left four suspects dead city across border mcallen texas used cartel violence key trafficking point victims saturday attacks appeared tamaulipas gov',
 'francisco garc cabeza de vaca called innocent citizens rather members one gang killed rival local businessman misael chavarria garza said many businesses closed early saturday attacks people scared helicopters flew overhead',
 'sunday said people quiet nothing happened feeling anger crime happened innocent people fair said taxi driver rene guevara adding among dead two fellow taxi drivers defended said involved crime attacks took place several neighborhoods eastern reynosa according tamaulipas state agency coordinates security forces sparked deployment military national guard state police across city',
 'images posted social media 

In [18]:
len(corpus)

11

In [14]:
n=300
all_words = [i.split() for i in corpus]
model = Word2Vec(all_words, min_count=1,vector_size= n)

In [16]:
model

In [19]:
sen_vector=[]
for i in corpus:
    
    plus=0
    for j in i.split():
        plus+=model.wv[j]
    plus = plus/len(plus)
    
    sen_vector.append(plus)

len(sen_vector)

11

In [22]:
n_clusters = 2
kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(sen_vector)

In [23]:
my_list=[]
for i in range(n_clusters):
    my_dict={}
    
    for j in range(len(y_kmeans)):
        
        if y_kmeans[j]==i:
            my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sen_vector[j])
    min_distance = min(my_dict.values())
    my_list.append(min(my_dict, key=my_dict.get))

In [25]:
my_list

[10, 5]

In [26]:
y_kmeans

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
summ =""
for i in sorted(my_list):
    summ+=sentence[i]

In [30]:
summ 

'Apparently, one cell from a nearby town may have entered Reynosa to carry out the attacks.Olga Ruiz, whose 19-year-old brother Fernando Ruiz was killed by the gunmen, said her sibling was working as a plumber and bricklayer in a company owned by his stepfather to pay for his studies.“They killed him in cold blood, he and two of his companions,” said Olga Ruiz, adding that the gunmen arrived where her brother was fixing a drain.“They heard the gunshots from afar and my stepfather told him: ‘son, you have to take shelter.’ So he asked permission to enter a house but my brother and his companions were only about to enter when the vehicles arrived,” Ruiz said.One former governor, Tomás Yarrington, was extradited to the United States from Italy in 2018 on drug trafficking charges.'

In [31]:
!pip install rouge -q

In [32]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(summ, df['Summary'][0] , avg = True)
RougeScore = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
RougeScore

,rouge-1,rouge-2,rouge-l
recall,0.236842,0.022727,0.157895
precision,0.093750,0.007353,0.062500
f-measure,0.134328,0.011111,0.089552


## On complete Dataset

In [33]:
df

,Article,Summary
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi..."
4,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N..."
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar..."
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...


In [56]:
df.drop(4,inplace = True)

In [57]:
df

,Article,Summary
0,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...
1,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,Representative ImageA 38-year-old Indian man h...,A 38-year-old Indian man has been charged with...
3,Residents get tested during their stay at a te...,"China on Apr 21 reported 2,119 locally transmi..."
5,Indian couple believed to be among those kille...,The daughter and son-in-law of Satish Deopujar...
...,...,...
4482,FILE - Tanks storing treated radioactive water...,A massive earthquake and tsunami in 2011 destr...
4483,Villagers clear debris caused by an earthquake...,"Over 680 tourists from France, Thailand, the N..."
4484,"Seif al-Islam, the son and one-time heir appar...","Seif al-Islam, the son and one-time heir appar..."
4485,Thai BoysMost members of the Thai youth footba...,Most members of the Thai youth football team r...


In [58]:
summaries = []
k=0
for text in df['Article']:
    print(k)
    k+=1
    sentence = sent_tokenize(text)
    corpus = []
    for i in range(len(sentence)):
        sen = re.sub('[^a-zA-Z]', " ", sentence[i])  
        sen = sen.lower()                            
        sen=sen.split()                         
        sen = ' '.join([i for i in sen if i not in stopwords.words('english')])   
        corpus.append(sen)
        
    n=300
    all_words = [i.split() for i in corpus]
    model = Word2Vec(all_words, min_count=1,vector_size= n)
    sen_vector=[]
    for i in corpus:
        plus=0
        for j in i.split():
            plus+=model.wv[j]
        plus = plus/len(plus)
        sen_vector.append(plus)
    n_clusters = 2
    kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
    y_kmeans = kmeans.fit_predict(sen_vector)
    my_list=[]
    for i in range(n_clusters):
        my_dict={}

        for j in range(len(y_kmeans)):

            if y_kmeans[j]==i:
                my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sen_vector[j])
        min_distance = min(my_dict.values())
        my_list.append(min(my_dict, key=my_dict.get))
    summ =""
    for i in sorted(my_list):
        summ+=sentence[i]
    summaries.append(summ)

0
1
2
3
4


ValueError: n_samples=1 should be >= n_clusters=2.

In [51]:
len(summaries)

4

In [52]:
sums = [df['Summary'][i] for i in range(4)]
sums

['The attacks took place in several neighborhoods in eastern Reynosa, according to the Tamaulipas state agency that coordinates security forces, and sparked a deployment of the military, National Guard and state police across the city. Images posted on social media showed bodies in the streets.',
 'Siddak Singh Jhamat, known as Sid, was using a fossil-hunting set he received as a Christmas present when he came across a rock that looked like a horn.',
 'A 38-year-old Indian man has been charged with conspiring to smuggle six of his countrymen into the US on commercial airline flights, authorities said.',
 'China on Apr 21 reported 2,119 locally transmitted confirmed COVID cases, of which 1,931 were reported in Shanghai, according to National Health Commission.']

In [53]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(summaries, sums , avg = True)
RougeScore = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
RougeScore

,rouge-1,rouge-2,rouge-l
recall,0.477154,0.239121,0.425261
precision,0.092295,0.019968,0.075592
f-measure,0.146908,0.034968,0.121788
